In [1]:
# Import de bibliotecas
import csv, ast, psycopg2
import pandas as pd
from datetime import datetime as dt
import numpy as np


In [2]:
# Lê dados de excel para dataframe
reader = pd.read_csv('BD2.csv',dtype="string", sep=';',header=0)#, engine='openpyxl')
# Confirma se todas as colunas foram importadas como string
print(reader.dtypes)

# Cria variáveis para armazenar cabeçalhos, tipos e tamanhos
longest, headers, type_list = [], [], []
# Cria variável com o nome da tabela que será criada
table_name = 'XPTO'
# Imprime amostra de dados do arquivo na tela
reader.head()

nota_fiscal     string
codigo_barra    string
descricao       string
qtd             string
data_compra     string
dia             string
mes             string
ano             string
preco           string
id_cliente      string
pais            string
data_compra2    string
data_compra3    string
data_compra4    string
dtype: object


,nota_fiscal,codigo_barra,descricao,qtd,data_compra,dia,mes,ano,preco,id_cliente,pais,data_compra2,data_compra3,data_compra4
0,0539993,22386,JUMBO BAG PINK POLKADOT,10,04/01/2011 10:00,4,1,2011,1.95,13313,United Kingdom,2011-01-04 10:00:00,2011-01-04,04/01/2011
1,539993,21499,BLUE POLKADOT WRAP,25,04/01/2011 10:00,4,1,2011,0.42,13313,United Kingdom,2011-01-04 10:00:00,2011-01-04,04/01/2011
2,539993,21498,RED RETROSPOT WRAP,25,04/01/2011 10:00,4,1,2011,0.42,13313,United Kingdom,2011-01-04 10:00:00,2011-01-04,04/01/2011
3,539993,22379,RECYCLING BAG RETROSPOT,5,04/01/2011 10:00,4,1,2011,2.1,13313,United Kingdom,2011-01-04 10:00:00,2011-01-04,04/01/2011
4,539993,20718,RED RETROSPOT SHOPPER BAG,10,04/01/2011 10:00,4,1,2011,1.25,13313,United Kingdom,2011-01-04 10:00:00,2011-01-04,04/01/2011


In [3]:
# Define função para análise dos tipos das colunas
# val representa o valor do campo e current_type o tipo do dado até o presente momento da análise
def dataType(val, current_type):
    # A primeira checagem consiste na verificação do tipo data (O algoritmo criado só trabalha com data completa)
    try:
        format = "%Y-%m-%d %H:%M:%S"
        date = dt.strptime(val, format)
        return 'timestamp'
    except:
        try:
            # A segunda checagem avalia números para o tipo apropriado
            t = ast.literal_eval(val)
        # Um erro na segunda checagem atribui o tipo varchar    
        except ValueError:
            return 'varchar'
        except SyntaxError:
            return 'varchar'
        # Caso a função não seja retornada por um try-catch
        # Se o tipo estiver entre int, long ou float    
        if type(t) in [int, 'long', float]:
            # Caso o tipo seja long ou int
            if (type(t) in [int, 'long']) and current_type not in ['float', 'varchar']:
                # Caso o primeiro caracter seja 0, atribui o tipo varchar, como identificadores de variáveis e CPFs iniciados com 0
                if val[0] == '0':
                    return 'varchar'
                # Caso contrário, atribui o menor valor possível como tipo (smallint, int ou bigint)
                elif (-32768 < t < 32767) and current_type not in ['int', 'bigint']:
                    return 'smallint'
                elif (-2147483648 < t < 2147483647) and current_type not in ['bigint']:
                    return 'int'
                else:
                    return 'bigint'
            # Valida o tamanho do varchar
            if type(t) is float and current_type not in ['varchar']:
                return 'decimal'
        # Se não estiver entre int, long ou float atribui varchar            
        else:
            return 'varchar'

In [4]:
# Para cada linha nos dados lidos
for x in range(len(reader)):
    # Captura apenas a linha em questão
    row = list(reader.iloc[x])
    # Se ainda não houver ocorrido a análise do cabeçalho (tamanho de headers == 0)
    if len(headers) == 0:
        # headers = linha atual
        headers = reader.columns
        # para todas as colunas na linha, o tamanho recebe 0 e o tipo recebe ''
        for col in headers:
            longest.append(0)
            type_list.append('')
    # Para cada coluna na linha
    for i in range(len(row)):
        # Caso já tenha sido definido como varchar, timestamp ou possua valor NA, nada é feito
        # NA é referente ao valor null no csv
        if type_list[i] == 'varchar' or str(row[i]) == 'NA' or type_list[i] == 'timestamp':
            pass
        # Caso contrário
        else:
            # Chama função para determinar tipos das colunas
            type_list[i] = dataType(str(row[i]), type_list[i])
        # Atribui tamanhos para as variáveis
        if len(str(row[i])) > longest[i]:
            longest[i] = len(str(row[i]))

In [5]:
# Cria variável para armazenar o codigo de criação da tabela
statement = 'create table ' + table_name + '('

# Escreve o nome e tipo variável por variável no código
for i in range(len(headers)):
    if type_list[i] == 'varchar':
        statement = (statement + '\n{} varchar({}),').format(headers[i].lower(), str(longest[i]))
    else:
        statement = (statement + '\n' + '{} {}' + ',').format(headers[i].lower(), type_list[i])
# Finaliza código com fechamento de parêntesis e ponto e vírgula
statement = statement[:-1] + ');'
# Imprime o código gerado
print(statement)

create table XPTO(
nota_fiscal varchar(7),
codigo_barra varchar(12),
descricao varchar(35),
qtd int,
data_compra varchar(16),
dia smallint,
mes smallint,
ano smallint,
preco varchar(7),
id_cliente varchar(6),
pais varchar(20),
data_compra2 timestamp,
data_compra3 varchar(10),
data_compra4 varchar(10));
